## Import libraries and dependencies


In [5]:
import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da
import optimiseprime.forecast as forecast
from optimiseprime.MCForecastTools import MCSimulation
import seaborn as sns

import datetime as dt

import pandas as pd
import yfinance as yf
import quantstats as qs
qs.extend_pandas()
import numpy as np

# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
#import hvplot.pandas
import matplotlib.pyplot as plt
%matplotlib inline

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [6]:
portfolio_choice = 0
portfolio_value = 0

while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio, type 2\n"
        )
                              )
    # Print error message if type(portfolio_choice) != int                        
    except ValueError:
        print('Error: Invalid response.\n')
        
    # Print error message if portfolio choice not 1 or 2
    if portfolio_choice not in [1, 2]:
        print('Error: Invalid response.\n')

# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        
        # Create a list of only the tickers
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
        
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
        
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")


To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio, type 2
 2


Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  BTC
Ticker:  ETH
Ticker:  done
How much do you wish to invest in total?
(Please input amount without currency symbol)
 30000


In [ ]:
# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)
display(portfolio_df)

In [ ]:
# Keep only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in list(portfolio_df.columns.levels[0])]
value_list = []
weight_list = []

# Print portfolio data for visual confirmation
def check (portfolio_choice, ticker_list, portfolio_value):
    if portfolio_choice == 1:   
        print(f"--------------------------\n")
        print(f"Hypothetical Portfolio:\n")
    # Calculate portfolio value of each cryptocurrency held
        for ticker in existing_portfolio:
            existing_portfolio[ticker].append(
            {'value': portfolio_df[f"{ticker}-USD"].iloc[-1, 3] * existing_portfolio[ticker][0]['units']})
            value = existing_portfolio[ticker][1]['value']
            portfolio_value += value
            print(f"Value of {existing_portfolio[ticker][0]['units']} {ticker}: ${value:.2f}")
            value_list.append(value)
        print(f"Total value of portfolio is ${portfolio_value:.2f}")
        pie_df = px.data.tips()
        px.pie(pie_df, values=value_list, names=ticker_list,   title="distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu).show()
    
        #find current weights
        for ticker in ticker_list:
            weight = (value_list[ticker_list.index(ticker)] / portfolio_value)
            weight_list.append(weight)
            print("Crypto Weights")
            print(f"The portfolio holds {weight*100:.2f}% {ticker}")
    
    elif portfolio_choice == 2:
        print(
            f"--------------------------\n"
            f"Hypothetical Portfolio:\n"
            f"{[ticker.replace('-USD', '') for ticker in ticker_list]}\n"                   
            f"Investment amount:\n"
            f"${portfolio_value:.2f}\n")
    print(
        f"NOTE:\n"
        f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
        f"While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.\n"
        f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
    )
    print(f"--------------------------")
    return portfolio_value

portfolio_value = check(portfolio_choice, ticker_list, portfolio_value)


      
    



In [ ]:
#weights of original
if portfolio_choice == 1:
    weight_og = pd.DataFrame([weight_list], columns = ticker_list)
    display(weight_og)

## Data Analyses

* Analysis of daily returns

In [ ]:
daily_returns = portfolio_df.filter(like="daily_return")
dr = []
for ticker in ticker_list:
    dr.append(ticker)
daily_returns.columns = dr
daily_returns

In [ ]:
#FIRST PAGE 
title = "Analysis of Daily Returns of Cryptos"
welcome_text = "In order to determine the sharpe ratio, the daily returns need to be known. \n Here the returns can be visually analysed to view returns and risk" 
daily_returns_plot = px.line(daily_returns, x = daily_returns.index, y = daily_returns.columns, title = "daily returns (volatility)")
first_page = pn.Column(title, welcome_text, daily_returns.head(), daily_returns_plot)

#SECOND PAGE
cumulative_returns = (1 + daily_returns).cumprod() - 1
cumul_plot = px.line(cumulative_returns, x = cumulative_returns.index, y = cumulative_returns.columns, title = "Cumulative Returns")

second_page = pn.Column("cumulative returns overtime", cumulative_returns.head(), cumul_plot)

#THIRD PAGE
boxplot = px.box(daily_returns, y=daily_returns.columns, title = 'Visual Representation of Risk')
third_page = pn.Column("Risk visualised on a box plot", boxplot)

#Fourth Page
if portfolio_choice == 1:
    portfolio_og = daily_returns.dot(weight_og.iloc[0,:])
    portfolio_og = pd.DataFrame(portfolio_og, columns = ['My Portfolio'])

    cumulative_returns_port = (1 + portfolio_og).cumprod() - 1
    daily_returns_port = px.line(portfolio_og, x = portfolio_og.index, y = portfolio_og.columns, title = "daily returns of my portfolio")
    cumul_plot_port = px.line(cumulative_returns_port, x = cumulative_returns_port.index, y = cumulative_returns_port.columns, title = "Cumulative Returns of my portfolio")

    fourth_page = pn.Column("Portfolio Analysis", portfolio_og.head(), daily_returns_port, cumul_plot_port)

if portfolio_choice == 1:
    daily_dashboard = pn.Tabs(
        ("Daily Returns of tickers", first_page),
        ("Cumulative Returns of tickers", second_page),
        ("Risk of tickers", third_page),
        ("Portfolio Analysis", fourth_page)
    )
else:
    daily_dashboard = pn.Tabs(
    ("Daily Returns of tickers", first_page),
    ("Cumulative Returns of tickers", second_page),
    ("Risk of tickers", third_page)
)


daily_dashboard.show()

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [ ]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adj_sortino': adjusted_sortino,
    'gain_pain': gain_pain_ratio,
    }
)
#display(ratios_df)

# Calculate proportion scores for each risk-reward metric
weights = da.calculate_weights(ratios_df)

# If user has selected to analyse an existing portfolio, add user's portfolio weights as well.
if portfolio_choice == 1 :
    weight_og = weight_og.transpose()
    weights = weights.merge(weight_og, left_index=True, right_index=True)
weights=weights.rename(columns = {0:'user_portfolio'})
weights



In [ ]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order

for column in ratios_df:
    if column == 'sharpe':
        da.sharpe_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'sortino':
        da.sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'adj_sortino':
        da.adj_sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'gain_pain':
        da.gain_pain_portfolio(ratios_df, weights, portfolio_value)
        
if portfolio_choice == 1:
    user_portfolio = da.user_portfolio(weights, portfolio_value)


In [ ]:

weights = weights.transpose()
weights

In [ ]:
names = weights.values.tolist()

sharpe_pie = px.pie(weights, 
    names = ticker_list,
    values = names[0],
    title = "Sharpe Recommended Weights")

sortino_pie =px.pie(weights, 
    names = ticker_list,
    values = names[1],
    title = "Sortino Recommended Weights")

adj_sortino_pie = px.pie(weights, 
    names = ticker_list,
    values = names[0],
    title = "Adj Sortino Recommended Weights")

pain_gain_pie = px.pie(weights, 
    names = ticker_list,
    values = names[1],
    title = "Pain Gain Recommended Weights")


if portfolio_choice == 1:
        px.pie(pie_df, values=value_list, names=ticker_list,   title="distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu)


In [ ]:
weights = weights.transpose()
weights

In [ ]:
ticker_close_names = []
for ticker in list(weights.index):
    ticker_close_names.append((ticker, 'close'))    
portfolio_close_df = portfolio_df[ticker_close_names]

portfolio_close_df

In [ ]:
portfolio_returns_list = []
for column_name in weights.columns:
    weights_list = weights[column_name].tolist()
    portfolio_returns = portfolio_close_df.dot(weights_list)
    portfolio_returns = portfolio_returns.rename(column_name)
    portfolio_returns_list.append(portfolio_returns)

port_returns_df = pd.DataFrame(portfolio_returns_list)
port_returns_df = port_returns_df.transpose()
port_returns_df

In [ ]:
port_returns = portfolio_close_df.pct_change()
port_returns = port_returns.dropna()

In [ ]:
port_returns.columns = dr
port_returns = port_returns.dot(weights)
port_returns
#daily returns

In [ ]:
true_false = pn.widgets.Select(name='View whole graph', options=[True, False])

    
interact = pn.interact(
    forecast.forecast_all_portfolios, 
    portfolio_returns_list = [portfolio_returns_list[0:5]], 
    window = range(365), 
    foward_looking = range(365), 
    full_picture = true_false
)




In [ ]:
#Set number of simulations 
num_sims = 500
num_trading_days = 365 * (int(input("Key in number of years to forecast:")))

MC_sharpe = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sharpe'].tolist(),
    num_trading_days = num_trading_days
)

MC_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_adj_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['adj_sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_gain_pain = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['gain_pain'].tolist(),
    num_trading_days = num_trading_days
)


simulations_list = [MC_sharpe, MC_sortino, MC_adj_sortino, MC_gain_pain]
if portfolio_choice == 1:
    MC_user = MCSimulation(
        portfolio_data = portfolio_df,
        num_simulation = num_sims,
        weights = weights['user_portfolio'].tolist(),
        num_trading_days = num_trading_days
    )
    simulations_list.append(MC_user)

for simulation in simulations_list:
    simulation.calc_cumulative_return()

In [ ]:
# Compute summary statistics from the simulated daily returns, for each simulation

all_cumulative_pnl = []
for simulation in simulations_list:
    simulated_returns = pd.DataFrame(
        {
            "mean": list(simulation.simulated_return.mean(axis=1)),
            "median": list(simulation.simulated_return.median(axis=1)),            
            "min": list(simulation.simulated_return.min(axis=1)),
            "max": list(simulation.simulated_return.max(axis=1))
        }
    )
    all_cumulative_pnl.append(simulated_returns)

# Concatenate cumulative pnls predicted by MC simulations for each strategy
all_mean_cumulative_pnl = pd.concat([df['mean'] for df in all_cumulative_pnl], join = 'inner', axis = 'columns') 
all_mean_cumulative_pnl = all_mean_cumulative_pnl * portfolio_value
#for df in all_cumulative_pnl:
    #data_mean = pd.DataFrame(df['mean'])
    #all_mean_cumulative_pnl = pd.concat([data_mean], axis = 'columns', join = 'inner')
all_mean_cumulative_pnl.columns = weights.columns.tolist()


In [ ]:
sim_port_mean = px.line(
    all_mean_cumulative_pnl,
    title = f"Simulated mean portfolio values over the next {num_trading_days / 365} years by R-R strategies",
    labels = {
        "value": "Portfolio Value ($)",
        "index": "Number of Trading Days"
        
    }
)


In [ ]:
#FIRST PAGE  - what was done, how
title = "Analysis of Portfolios"
welcome_text = "What was done and how" 
first_page = pn.Column(title, welcome_text)

#SECOND PAGE - returns of tickers
daily_returns = round(daily_returns, 2)
cumulative_returns = (1 + daily_returns).cumprod() - 1
cumul_plot = px.line(cumulative_returns, x = cumulative_returns.index, y = cumulative_returns.columns, title = "Cumulative Returns of Tickers")
daily_returns_plot = px.line(daily_returns, x = daily_returns.index, y = daily_returns.columns, title = "Daily Returns of Tickers")
boxplot = px.box(daily_returns, y=daily_returns.columns, title = 'Risk of Tickers')

second_page = pn.Column("Ticker Analysis", daily_returns.head(), daily_returns_plot, cumulative_returns.head(), cumul_plot)

#THIRD PAGE (if) - og port analysis
if portfolio_choice ==1:
    pie_og = px.pie(pie_df, values=value_list, names=ticker_list, title="distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu)
    third_page = pn.Column("Original Portfolio Analysis", weight_og, pie_og, portfolio_og.head(), daily_returns_port, cumul_plot_port, boxplot)
        
#Fourth Page
if portfolio_choice == 1:
    fourth_page = pn.Column("Recommended Weights in Comparison to Original Portfolio",  weight_og, pie_og, weights, sharpe_pie, sortino_pie, adj_sortino_pie, pain_gain_pie)
else:
    fourth_page = pn.Column("Recommended Weights for Portfolio", weights, sharpe_pie, sortino_pie, adj_sortino_pie, pain_gain_pie)
        
#Fifth Page
port_cumulative_returns = (1 + port_returns).cumprod() - 1
port_cumul_plot = px.line(port_cumulative_returns, x = port_cumulative_returns.index, y = port_cumulative_returns.columns, title = "Cumulative Returns of Tickers")
port_returns_plot = px.line(port_returns, x = port_returns.index, y = port_returns.columns, title = "Daily Returns of Portfolio")
port_boxplot = px.box(port_returns, y=port_returns.columns, title = 'Risk of Portfolio')

fifth_page = pn.Column("Portfolio Analysis", "Portfolio Returns", port_returns.head(), port_returns_plot, "Portfolio Cumulative Returns",port_cumulative_returns.head(), port_cumul_plot, "Visualised Risk of Each Portfolio", port_boxplot)

#sixth
ann_std = port_returns.std() * np.sqrt(252)
# Calculate rolling standard deviation
r21_std = port_returns.rolling(window=21).std()
r21_std_plot = px.line(r21_std, x = r21_std.index, y = r21_std.columns, title = "Rolling-21 Day Std")

correlation = port_returns.corr()
corr_plot = px.imshow(correlation)

sixth_page = pn.Column("Annual Standard Deviation", ann_std, r21_std_plot, "Correlation of portfolios", correlation, corr_plot, "Portfolio Close", portfolio_close_df.tail(), "Portfolio Returns", port_returns_df.tail(), interact[1], interact[0])

seventh_page = pn.Column("Simulated Returns", simulated_returns.tail(), "Mean Cumulative PNL", all_mean_cumulative_pnl.tail(), sim_port_mean)


if portfolio_choice == 1:
    final_dashboard = pn.Tabs(
        ("Daily Returns of tickers", first_page),
        ("Cumulative Returns of tickers", second_page),
        ("Risk of tickers", third_page),
        ("Recommended Weights", fourth_page),
        ("Comparing Portfolios", fifth_page),
        ("Comparing Portfolios" , sixth_page),
        ("Monte Carlo" , seventh_page)
    )
else:
    final_dashboard = pn.Tabs(
    ("Welcome", first_page),
    ("Ticker Analysis", second_page),
    ("Recommended Weights", fourth_page),
    ("Comparing Portfolios", fifth_page),
    ("Comparing Portfolios" , sixth_page),
    ("Monte Carlo" , seventh_page)
)

final_dashboard.show()
    
